<a href="https://colab.research.google.com/github/aworklab/everydayPython/blob/main/ex_20230104.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 오늘의 목표
* Customer Analysis
* Iterators

In [1]:
import pandas as pd
import numpy as np
import plotnine as gg
from plotnine import ggplot, aes
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, date, time, timezone, timedelta
import os
import re

from functools import reduce
from collections import defaultdict 

# Customer Analysis

In [2]:
os.listdir()

['.config',
 '1_order_payments_dataset.csv',
 'KPMG_VI_New_raw_data_update_final.csv',
 'ecommerce-data.csv',
 'sales_data_with_stores.csv',
 'match_result_test.csv',
 'bz_data_analyis.csv',
 'mock_sales_data.csv',
 'mall_customers.csv',
 'WA_Fn-UseC_-Telco-Customer-Churn.csv',
 'online_retail.xlsx',
 'test.csv',
 'KPMG_VI_New_raw_data_update_final.xlsx',
 '1_customers_dataset.csv',
 'WA_Fn-UseC_-Marketing-Customer-Value-Analysis.csv',
 'online_retail_II.xlsx',
 'sales.csv',
 'sales_vs_discount.csv',
 'scanner_data.csv',
 '1_orders_dataset.csv',
 'groceries.csv',
 'sample-sales-data.csv',
 'sample_data']

In [5]:
df = pd.read_csv('ecommerce-data.csv', encoding = 'unicode_escape')
df.columns = ['_'.join(re.split('(?=[A-Z][a-z])', a[0].lower() + a[1:])).lower() for a in df.columns]
df['invoice_date'] = pd.to_datetime(df['invoice_date'])
df['date'] = df['invoice_date'].to_numpy().astype('datetime64[D]')
df['week'] = df['date'] - df['date'].dt.weekday.astype('timedelta64[D]')
df['rev'] = df.quantity * df.unit_price
df.head()

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,customerid,country,date,week,rev
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,2010-12-01,2010-11-29,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01,2010-11-29,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,2010-12-01,2010-11-29,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01,2010-11-29,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01,2010-11-29,20.34


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 11 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   invoice_no    541909 non-null  object        
 1   stock_code    541909 non-null  object        
 2   description   540455 non-null  object        
 3   quantity      541909 non-null  int64         
 4   invoice_date  541909 non-null  datetime64[ns]
 5   unit_price    541909 non-null  float64       
 6   customerid    406829 non-null  float64       
 7   country       541909 non-null  object        
 8   date          541909 non-null  datetime64[ns]
 9   week          541909 non-null  datetime64[ns]
 10  rev           541909 non-null  float64       
dtypes: datetime64[ns](3), float64(3), int64(1), object(4)
memory usage: 45.5+ MB


In [12]:
flag_invoice_no = df[['invoice_no']].assign(flag = ['o' if re.findall('[A-Z]|[a-z]', str(a)) else 'x' for a in df['invoice_no']] )
flag_invoice_no[flag_invoice_no['flag'] == 'o']

,invoice_no,flag
141,C536379,o
154,C536383,o
235,C536391,o
236,C536391,o
237,C536391,o
...,...,...
540449,C581490,o
541541,C581499,o
541715,C581568,o
541716,C581569,o


In [14]:
df['flag_invoice'] = ['o' if re.findall('[A-Z]|[a-z]', str(a)) else 'x' for a in df['invoice_no']]
df.head()

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,customerid,country,date,week,rev,flag_invoice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,2010-12-01,2010-11-29,15.30,x
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01,2010-11-29,20.34,x
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,2010-12-01,2010-11-29,22.00,x
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01,2010-11-29,20.34,x
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01,2010-11-29,20.34,x


In [17]:
df['description'].value_counts().reset_index()

,index,description
0,WHITE HANGING HEART T-LIGHT HOLDER,2369
1,REGENCY CAKESTAND 3 TIER,2200
2,JUMBO BAG RED RETROSPOT,2159
3,PARTY BUNTING,1727
4,LUNCH BAG RED RETROSPOT,1638
...,...,...
4218,Missing,1
4219,historic computer difference?....se,1
4220,DUSTY PINK CHRISTMAS TREE 30CM,1
4221,WRAP BLUE RUSSIAN FOLKART,1


In [20]:
df[['description', 'flag_invoice']].value_counts().reset_index()

,description,flag_invoice,0
0,WHITE HANGING HEART T-LIGHT HOLDER,x,2327
1,JUMBO BAG RED RETROSPOT,x,2115
2,REGENCY CAKESTAND 3 TIER,x,2019
3,PARTY BUNTING,x,1707
4,LUNCH BAG RED RETROSPOT,x,1594
...,...,...,...
6174,T-LIGHT HOLDER HANGING LACE,o,1
6175,BAROQUE BUTTERFLY EARRINGS RED,x,1
6176,DECORATION WOBBLY CHICKEN,o,1
6177,Sold as 1 on dotcom,x,1


In [21]:
df[df['flag_invoice'] == 'o'][['description']].value_counts().reset_index()

,description,0
0,Manual,244
1,REGENCY CAKESTAND 3 TIER,181
2,POSTAGE,126
3,JAM MAKING SET WITH JARS,87
4,Discount,77
...,...,...
1968,MAGIC SHEEP WOOL GROWING FROM PAPER,1
1969,MAGIC TREE -PAPER FLOWERS,1
1970,MAGNETS PACK OF 4 HOME SWEET HOME,1
1971,MAKE YOUR OWN FLOWERPOWER CARD KIT,1


In [24]:
cnt = df.groupby(['customerid', 'week'])['rev'].agg(['sum','count']).reset_index()
cnt.head()

,customerid,week,sum,count
0,12346.0,2011-01-17,0.00,2
1,12347.0,2010-12-06,711.79,31
2,12347.0,2011-01-24,475.39,29
3,12347.0,2011-04-04,636.25,24
4,12347.0,2011-06-06,382.52,18


# Iterators

In [28]:
users = [{'mail': 'gregorythomas@gmail.com', 'name': 'Brett Holland', 'sex': 'M', 'age': 73},
 {'mail': 'hintoncynthia@hotmail.com', 'name': 'Madison Martinez', 'sex': 'F', 'age': 29},
 {'mail': 'wwagner@gmail.com', 'name': 'Michael Jenkins', 'sex': 'M', 'age': 51},
 {'mail': 'daniel79@gmail.com', 'name': 'Karen Rodriguez', 'sex': 'F', 'age': 32},
 {'mail': 'ujackson@gmail.com', 'name': 'Amber Rhodes', 'sex': 'F', 'age': 42}]
users

[{'mail': 'gregorythomas@gmail.com',
  'name': 'Brett Holland',
  'sex': 'M',
  'age': 73},
 {'mail': 'hintoncynthia@hotmail.com',
  'name': 'Madison Martinez',
  'sex': 'F',
  'age': 29},
 {'mail': 'wwagner@gmail.com',
  'name': 'Michael Jenkins',
  'sex': 'M',
  'age': 51},
 {'mail': 'daniel79@gmail.com',
  'name': 'Karen Rodriguez',
  'sex': 'F',
  'age': 32},
 {'mail': 'ujackson@gmail.com', 'name': 'Amber Rhodes', 'sex': 'F', 'age': 42}]

In [29]:
{k : eval(f'reduce(lambda acc, cur : acc + [cur["{k}"]], users, [])') for k in list(users[0].keys())}

{'mail': ['gregorythomas@gmail.com',
  'hintoncynthia@hotmail.com',
  'wwagner@gmail.com',
  'daniel79@gmail.com',
  'ujackson@gmail.com'],
 'name': ['Brett Holland',
  'Madison Martinez',
  'Michael Jenkins',
  'Karen Rodriguez',
  'Amber Rhodes'],
 'sex': ['M', 'F', 'M', 'F', 'F'],
 'age': [73, 29, 51, 32, 42]}